In [1]:
import pandas as pd
from datetime import datetime
import sqlite3

In [2]:
# https://fftoday.com/nfl/schedule.php

In [3]:
df = pd.read_csv("schedule.csv", skiprows=1)

In [4]:
df['Date'] = df['Date'].fillna(method='ffill')

/tmp/ipykernel_538544/1291857192.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Date'] = df['Date'].fillna(method='ffill')


In [5]:
df

,Date,Time (ET),Away Team,Home Team
0,Thu Sep 5,8:20 pm,Baltimore Ravens,Kansas City Chiefs
1,Fri Sep 6,8:15 pm,Green Bay Packers,Philadelphia Eagles ¹
2,Sun Sep 8,1:00 pm,Arizona Cardinals,Buffalo Bills
3,Sun Sep 8,1:00 pm,Carolina Panthers,New Orleans Saints
4,Sun Sep 8,1:00 pm,Houston Texans,Indianapolis Colts
...,...,...,...,...
312,Sun Jan 5,1:00 pm,New York Giants,Philadelphia Eagles *
313,Sun Jan 5,1:00 pm,San Francisco 49ers,Arizona Cardinals *
314,Sun Jan 5,1:00 pm,Seattle Seahawks,Los Angeles Rams *
315,Sun Jan 5,1:00 pm,Washington Commanders,Dallas Cowboys *


In [6]:
def parse_date(date_str):
    date = pd.to_datetime(date_str, format='%a %b %d', errors='coerce')
    current_year = datetime.now().year
    
    # If the month is January, set the year to next year
    if date.month == 1:
        return date.replace(year=current_year + 1)
    else:
        return date.replace(year=current_year)

df['Date'] = df['Date'].apply(parse_date)

In [7]:
df.dropna(subset=['Date'], inplace=True)

In [8]:
import re
# Remove ' ¹' or '*' from the end of the Home Team column
df['Home Team'] = df['Home Team'].str.replace(r'[ ¹*]+$', '', regex=True)

In [9]:
df['DateTime'] = pd.to_datetime(df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time (ET)'])

/tmp/ipykernel_538544/364424183.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DateTime'] = pd.to_datetime(df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time (ET)'])


In [10]:
df = df[['DateTime', 'Away Team', 'Home Team']].rename(columns={'DateTime': 'datetime', 'Away Team': 'away_team', 'Home Team': 'home_team'})

In [12]:
df = df.reset_index(names='game_id')

In [13]:
df

,game_id,datetime,away_team,home_team
0,0,2024-09-05 20:20:00,Baltimore Ravens,Kansas City Chiefs
1,1,2024-09-06 20:15:00,Green Bay Packers,Philadelphia Eagles
2,2,2024-09-08 13:00:00,Arizona Cardinals,Buffalo Bills
3,3,2024-09-08 13:00:00,Carolina Panthers,New Orleans Saints
4,4,2024-09-08 13:00:00,Houston Texans,Indianapolis Colts
...,...,...,...,...
267,311,2025-01-05 13:00:00,New Orleans Saints,Tampa Bay Buccaneers
268,312,2025-01-05 13:00:00,New York Giants,Philadelphia Eagles
269,313,2025-01-05 13:00:00,San Francisco 49ers,Arizona Cardinals
270,314,2025-01-05 13:00:00,Seattle Seahawks,Los Angeles Rams


In [14]:
df.to_parquet("schedule.parquet")

In [51]:
conn = sqlite3.connect('data/main.db')
df.to_sql('schedule', conn, if_exists='replace', index=True, index_label='game_id')
conn.close()

In [48]:
df

,datetime,away_team,home_team
0,2024-09-05 20:20:00,Baltimore Ravens,Kansas City Chiefs
1,2024-09-06 20:15:00,Green Bay Packers,Philadelphia Eagles
2,2024-09-08 13:00:00,Arizona Cardinals,Buffalo Bills
3,2024-09-08 13:00:00,Carolina Panthers,New Orleans Saints
4,2024-09-08 13:00:00,Houston Texans,Indianapolis Colts
...,...,...,...
311,2025-01-05 13:00:00,New Orleans Saints,Tampa Bay Buccaneers
312,2025-01-05 13:00:00,New York Giants,Philadelphia Eagles
313,2025-01-05 13:00:00,San Francisco 49ers,Arizona Cardinals
314,2025-01-05 13:00:00,Seattle Seahawks,Los Angeles Rams


In [2]:
import requests
import os
import pandas as pd

from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('ODDS_API_KEY')
r = requests.get(f"https://api.the-odds-api.com/v4/sports/americanfootball_nfl/scores/?daysFrom=3&apiKey={api_key}")
results = pd.DataFrame(r.json())

In [5]:
results

,id,sport_key,sport_title,commence_time,completed,home_team,away_team,scores,last_update
0,ddd9b67cf1d68282e1e8652000c0d015,americanfootball_nfl,NFL,2024-09-13T00:16:05Z,True,Miami Dolphins,Buffalo Bills,"[{'name': 'Buffalo Bills', 'score': '31'}, {'n...",2024-09-15T22:33:22Z
1,5a2bfc721fb2300d4578743aee7ad891,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Baltimore Ravens,Las Vegas Raiders,"[{'name': 'Las Vegas Raiders', 'score': '26'},...",2024-09-15T22:33:22Z
2,111ac41e21c6f16a2d3d1511f07e2004,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Jacksonville Jaguars,Cleveland Browns,"[{'name': 'Cleveland Browns', 'score': '18'}, ...",2024-09-15T22:33:22Z
3,787ae4144b2b45d7ef8a9c7b1ab6abd8,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Detroit Lions,Tampa Bay Buccaneers,"[{'name': 'Tampa Bay Buccaneers', 'score': '20...",2024-09-15T22:33:22Z
4,c4e5642a8746f3e8fbb09d59a611ffe4,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Green Bay Packers,Indianapolis Colts,"[{'name': 'Indianapolis Colts', 'score': '10'}...",2024-09-15T22:33:22Z
5,9d5dace601f10c135f5c03facf23fd22,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Minnesota Vikings,San Francisco 49ers,"[{'name': 'San Francisco 49ers', 'score': '17'...",2024-09-15T22:33:22Z
6,42d27889b51ada96c20671b30129ce4e,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,New England Patriots,Seattle Seahawks,"[{'name': 'Seattle Seahawks', 'score': '23'}, ...",2024-09-15T22:33:22Z
7,17d4d840f9f4094d76c6e9b53b18ac05,americanfootball_nfl,NFL,2024-09-15T17:02:39Z,True,Tennessee Titans,New York Jets,"[{'name': 'New York Jets', 'score': '24'}, {'n...",2024-09-15T22:33:22Z
8,5b05775b536cda3f2efab93935af4f54,americanfootball_nfl,NFL,2024-09-15T17:02:45Z,True,Dallas Cowboys,New Orleans Saints,"[{'name': 'New Orleans Saints', 'score': '44'}...",2024-09-15T22:33:22Z
9,495d9f2c316b05ef092099ef9401a180,americanfootball_nfl,NFL,2024-09-15T17:02:58Z,True,Washington Commanders,New York Giants,"[{'name': 'New York Giants', 'score': '18'}, {...",2024-09-15T22:33:22Z


In [9]:
# ... existing code ...

# Explode the 'scores' column to create separate rows for each team's score
results_exploded = results.explode('scores')

# Extract 'name' and 'score' from the 'scores' dictionary
results_exploded['team'] = results_exploded['scores'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None)
results_exploded['score'] = results_exploded['scores'].apply(lambda x: x['score'] if isinstance(x, dict) and 'score' in x else None)

# Drop the original 'scores' column
results_exploded = results_exploded.drop('scores', axis=1)

# Create separate columns for home and away scores
def get_scores(group):
    home_team = group['home_team'].iloc[0] if 'home_team' in group.columns and not group.empty else None
    away_team = group['away_team'].iloc[0] if 'away_team' in group.columns and not group.empty else None
    
    home_score = group[group['team'] == home_team]['score'].iloc[0] if home_team and not group[group['team'] == home_team].empty else None
    away_score = group[group['team'] == away_team]['score'].iloc[0] if away_team and not group[group['team'] == away_team].empty else None
    
    return pd.Series({'home_team_score': home_score, 'away_team_score': away_score})

results_fixed = results_exploded.groupby('id', as_index=False).apply(get_scores)

# Merge the scores back with the original data
results_fixed = results.merge(results_fixed, on='id', how='left')

# Reorder columns for better readability
column_order = ['id', 'sport_key', 'sport_title', 'commence_time', 'completed', 'home_team', 'away_team', 'home_team_score', 'away_team_score', 'last_update']
results_fixed = results_fixed[column_order]

# ... existing code ...

In [10]:
results_fixed

,id,sport_key,sport_title,commence_time,completed,home_team,away_team,home_team_score,away_team_score,last_update
0,ddd9b67cf1d68282e1e8652000c0d015,americanfootball_nfl,NFL,2024-09-13T00:16:05Z,True,Miami Dolphins,Buffalo Bills,10,31,2024-09-15T22:33:22Z
1,5a2bfc721fb2300d4578743aee7ad891,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Baltimore Ravens,Las Vegas Raiders,23,26,2024-09-15T22:33:22Z
2,111ac41e21c6f16a2d3d1511f07e2004,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Jacksonville Jaguars,Cleveland Browns,13,18,2024-09-15T22:33:22Z
3,787ae4144b2b45d7ef8a9c7b1ab6abd8,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Detroit Lions,Tampa Bay Buccaneers,16,20,2024-09-15T22:33:22Z
4,c4e5642a8746f3e8fbb09d59a611ffe4,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Green Bay Packers,Indianapolis Colts,16,10,2024-09-15T22:33:22Z
5,9d5dace601f10c135f5c03facf23fd22,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,Minnesota Vikings,San Francisco 49ers,23,17,2024-09-15T22:33:22Z
6,42d27889b51ada96c20671b30129ce4e,americanfootball_nfl,NFL,2024-09-15T17:02:00Z,True,New England Patriots,Seattle Seahawks,20,23,2024-09-15T22:33:22Z
7,17d4d840f9f4094d76c6e9b53b18ac05,americanfootball_nfl,NFL,2024-09-15T17:02:39Z,True,Tennessee Titans,New York Jets,17,24,2024-09-15T22:33:22Z
8,5b05775b536cda3f2efab93935af4f54,americanfootball_nfl,NFL,2024-09-15T17:02:45Z,True,Dallas Cowboys,New Orleans Saints,19,44,2024-09-15T22:33:22Z
9,495d9f2c316b05ef092099ef9401a180,americanfootball_nfl,NFL,2024-09-15T17:02:58Z,True,Washington Commanders,New York Giants,21,18,2024-09-15T22:33:22Z
